In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import tensorflow as tf

2024-07-10 08:16:51.884039: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Experimento com redes convolucionais

Neste experimento, iremos mostrar um reconhecedor de dígitos manuscritos. Para isto, carregaremos o dataset MNIST e treinaremos uma rede convolucional para executar a tarefa de classificação.

Para testar o sistema, iremos ter um "board" onde poderemos escrever um número para usar como entrada para rede e ver sua classificação.

- Aqui definimos a classe auxiliar que fará o board para escrevermos com o mouse. Essa classe é auxiliar e não faz parte do experimento.

In [ ]:
class Board:

    def __init__(self):

        self.b = None
        self.x = None
        self.y = None
        self.complete = False

        self.nx = 28
        self.ny = 28

        self.fig = plt.figure()
        self.img = np.zeros((self.nx, self.ny))
        self.im = plt.imshow(self.img, cmap='gist_gray_r', vmin=0, vmax=1)

        plt.connect('motion_notify_event', self.mouse_move)

        self.anim = animation.FuncAnimation(self.fig, self.animate, init_func=self.init, frames=self.nx*self.ny, interval=50)


    def mouse_move(self, event):

        self.x, self.y, self.b = event.xdata, event.ydata, event.button



    def setPixel(self, img, x, y, val):
        if x>=0 and y>=0 and x < img.shape[0] and y < img.shape[1]:
            img[x, y] += val
            if img[x, y] > 1:
                img[x, y] = 1


    def drawPoint(self, img, x, y):

        self.setPixel(img, x, y, 1)
        self.setPixel(img, x+1, y, 0.25)
        self.setPixel(img, x, y+1, 0.25)
        self.setPixel(img, x-1, y+1, 0.25)
        self.setPixel(img, x+1, y+1, 0.25)

        self.setPixel(img, x+1, y, 0.25)
        self.setPixel(img, x, y+1, 0.25)
        self.setPixel(img, x-1, y, 0.25)
        self.setPixel(img, x, y-1, 0.25)






    def init(self):
        self.im.set_data(np.zeros((self.nx, self.ny)))

    def animate(self, i):


        if self.b == plt.MouseButton.LEFT:
            self.drawPoint(self.img, int(self.y), int(self.x))

        if self.b == plt.MouseButton.RIGHT:
            self.complete = True


        self.im.set_data(self.img + np.random.randn(self.nx,self.ny)*0.05)
        return self.im


    def run(self):

        while not self.complete:
            plt.pause(0.1)

        plt.close()


### inicio do experimento propriamente dito

- carregamos o dataset utilizando o keras.dataset

In [ ]:
# 1. Carregar conjuntdo MNIST
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()


- preparamos o dataset para entrada na rede:

    Normalização e reshape de cada elemento do dataset

    Fazemos isso tanto para entrada quanto para saída

In [ ]:
# 2. Preprocessar entradas para ajustar os tamanhos
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)/255.0
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)/255.0


In [ ]:
# 3. Ajustar formato das saídas
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


- Aqui construimos o modelo

In [ ]:
# 4. construir o modelo
model = tf.keras.Sequential()

model.add( tf.keras.layers.Input((28,28,1)) )

model.add( tf.keras.layers.Convolution2D(9, (5, 5), padding = 'same', activation='relu') )

#model.add( tf.keras.layers.Convolution2D(16, (3, 3), padding = 'same', activation='relu') )

model.add( tf.keras.layers.Flatten() )

model.add( tf.keras.layers.Dense(10, activation='softmax') )


- Compilamos o mesmo, escolhendo uma função de loss, o algoritmo de optimização e uma métrica para avaliaçao

In [ ]:
# 5. Compilar o modelo
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


- Utilizando o sumario, podemos verificar o número de parâmetros envolvidos no modelo

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 9)      │           234 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7056)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        70,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,414 (829.75 KB)

 Trainable params: 70,804 (276.58 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 141,610 (553.17 KB)

- Aqui realizamos o treinamento. Como ja temos um modelo treinado, a linha fit esta comentada, e apenas carregamos o modelo ja treinado

In [ ]:
# 6. Treino!
#model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1)
model = tf.keras.models.load_model('mnist_trained.keras')


In [ ]:
model.save('mnist_trained.keras')

- Aqui testamos o modelo com várias imagens que não foram treinadas para validar o seu treinamento

In [ ]:
# 7. plotar alguns resultados...
fig = plt.figure(figsize=(14,10))
count = 1
for i in range(5):
    for j in range(5):
        plt.subplot(5,5,count)

        x = X_test[count].reshape((1,28,28,1))

        y = model.predict(x)

        plt.imshow((255*x).reshape(28,28).astype('uint8'))
        plt.title(np.argmax(y))
        plt.axis('off')

        count = count+1

plt.show()
plt.pause(0.001)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


- Finalmente vamos testar em uma aplicação real. O board irá permitir que escrevamos um digito com o mouse e a rede vai tentar classificar.

- Para terminar, pressione o botão direito do mouse e arraste um pouco

In [ ]:
%matplotlib qt5

def plotFiltered(model, img):

    y = model.layers[0](img.reshape((1,28,28,1)))

    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(y[0,:,:,i])
        plt.axis('off')
    plt.tight_layout(pad=0.4)


board = Board()
board.run()

y = np.argmax(model.predict(board.img.reshape(1,28,28,1)))

plt.figure()
plotFiltered(model, board.img)
print(f'prediction: {y}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
prediction: 3
